## Goal: intersect PLSs w/ TSSs called by LAPA for each sample

In [97]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [98]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [151]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df.loc[df.mixed_samples==False]
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample']].drop_duplicates()

In [152]:
pls_file = proc_cfg(config['ref']['ccre']['pls_bed'], od)
pls = pr.read_bed(pls_file)

In [153]:
bed_files = expand(proc_cfg(config['lr']['lapa']['sample_tss'], od),
                   sample=df['sample'].tolist())

In [154]:
f = bed_files[0]
f = '../data/lapa/sample/LWK1.bed'
tss = pr.read_bed(f)

tss = tss.df
tss.rename({'ItemRGB': 'tpm'}, axis=1, inplace=True)
tss['rid'] = tss['Chromosome'].astype(str)+'_'+\
             tss['Start'].astype(str)+'_'+\
             tss['Strand'].astype(str)+'_'+\
             tss['End'].astype(str)
tss = tss[['Chromosome', 'Start', 'End', 'Strand', 'tpm', 'rid']]
tss = tss.drop_duplicates()
l1 = len(tss.index)

In [155]:
# assert len(tss) == len(tss.rid.unique())
tss.loc[tss.rid.duplicated(keep=False)].sort_values(by='rid').head()

,Chromosome,Start,End,Strand,tpm,rid


In [156]:
tss = pr.PyRanges(tss)
tss = tss.join(pls,
               strandedness=None,
               how='left')

In [157]:
# deduplicate entries from original TSSs by whether the intersect
# at least one ccre 
tss = tss.df
tss['int_pls'] = False
tss.loc[tss.Start_b != -1, 'int_pls'] = True
gb_cols = ['Chromosome', 'Start', 'End', 'Strand', 'tpm', 'rid']
keep_cols = gb_cols+['int_pls']
tss = tss[keep_cols].groupby(gb_cols, observed=True).max().reset_index()

In [158]:
tss

,Chromosome,Start,End,tpm,rid,int_pls
0,GL000194.1,115043,115086,12.18,GL000194.1_115043_-_115086,False
1,GL000195.1,86661,86727,8.62,GL000195.1_86661_-_86727,False
2,GL000195.1,137913,138001,31.40,GL000195.1_137913_+_138001,False
3,GL000205.2,63600,63601,14.06,GL000205.2_63600_-_63601,False
4,GL000205.2,63674,63679,7.53,GL000205.2_63674_-_63679,False
...,...,...,...,...,...,...
8217,chrY,19567005,19567091,3.66,chrY_19567005_-_19567091,True
8218,chrY,19567311,19567380,12.78,chrY_19567311_+_19567380,True
8219,chrY,19567903,19567904,1.68,chrY_19567903_+_19567904,False
8220,chrY,20575690,20575798,277.72,chrY_20575690_+_20575798,True


In [92]:
# make sure there are no dupes
assert len(tss) == len(tss.rid.unique())
l2 = len(tss.index)
assert l1 == l2

AssertionError: 

In [93]:
l1

8222

In [94]:
l2

0